In [ ]:
import numpy as np
import verde as vd
import magali as mg
import harmonica as hm
import skimage.exposure

import xarray as xr

import matplotlib.pyplot as plt
from magali._utils import total_gradient_amplitude_grid

SEED = 5
rng = np.random.default_rng(SEED)

sensor_sample_distance = 5.0  # µm
region = [0, 2000, 0, 2000]  # µm
spacing = 2  # µm

true_inclination = 30  # degrees
true_declination = 40  # degrees
true_dispersion_angle = 5  # degrees
size = 100  # number of random dipoles

directions_inclination, directions_declination = mg.random_directions(
    true_inclination,
    true_declination,
    true_dispersion_angle,
    size=size,
    random_state= SEED,
)

dipoles_amplitude = abs(rng.normal(0, 100, size)) * 1.0e-14

dipole_coordinates = (
    rng.integers(30, 1970, size), # x
    rng.integers(30, 1970, size), # y
    rng.integers(-20, -1, size), # z
)

dipole_moments = hm.magnetic_angles_to_vec(
    inclination=directions_inclination,
    declination=directions_declination,
    intensity=dipoles_amplitude,
)

data = mg.dipole_bz_grid(
    region, spacing, sensor_sample_distance,
    dipole_coordinates, dipole_moments
)

noise_std_dev = 100  # nT
noise = rng.normal(loc=0, scale=noise_std_dev, size=data.shape)
data.values += noise

data_tga = total_gradient_amplitude_grid(data)
stretched = skimage.exposure.rescale_intensity(
    data_tga,
    in_range=tuple(np.percentile(data_tga, (1, 99))),
)
data_tga_stretched = xr.DataArray(stretched, coords=data_tga.coords)


windows = mg.detect_anomalies(
    data,
    size_range = [25, 50], #µm
    detection_threshold = 0.8,
    border_exclusion= 2,
)

fig, ax = plt.subplots()
data.plot.pcolormesh(ax=ax, cmap="seismic", vmin=-5000, vmax=5000)
mg.plot_bounding_boxes(windows, ax=ax, color="black", linewidth=1.5)


ax.set_title("Detected Anomalies")
plt.show()


/home/yago/miniforge3/envs/latinmag/lib/python3.9/site-packages/xrft/xrft.py:573: FutureWarning: Default ifft's behaviour (lag=None) changed! Default value of lag was zero (centered output coordinates) and is now set to transformed coordinate's attribute: 'direct_lag'.
  warnings.warn(msg, FutureWarning)
/home/yago/miniforge3/envs/latinmag/lib/python3.9/site-packages/xrft/xrft.py:573: FutureWarning: Default ifft's behaviour (lag=None) changed! Default value of lag was zero (centered output coordinates) and is now set to transformed coordinate's attribute: 'direct_lag'.
  warnings.warn(msg, FutureWarning)


NameError: name 'xr' is not defined